# CSIC 5011 Project1 - Animal Sleeping Data
Is there a linear model represent properties/features among animals?
<br>
If not, is it possible to get better result with hybrid linear model?

In [1]:
import sys
import pandas as pd
import numpy as np
import csv
import math
import matplotlib.pyplot as plt
%matplotlib notebook
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Button, Layout
from IPython.display import display

plt.close('all')

In [2]:
df = pd.read_csv('sleep1.csv', sep=',')
df

,species,slowWaveSleep,dreamSleep,sleep,body,brain,life,gestation,predation,sleepExposure,danger
0,African_elephant,NaN,NaN,3.3,6654.000,5712.00,38.6,645.0,3,5,3
1,African_giant_pouched_rat,6.3,2.0,8.3,1.000,6.60,4.5,42.0,3,1,3
2,Arctic_Fox,NaN,NaN,12.5,3.385,44.50,14.0,60.0,1,1,1
3,Arctic_ground_squirrel,NaN,NaN,16.5,0.920,5.70,NaN,25.0,5,2,3
4,Asian_elephant,2.1,1.8,3.9,2547.000,4603.00,69.0,624.0,3,5,4
5,Baboon,9.1,0.7,9.8,10.550,179.50,27.0,180.0,4,4,4
6,Big_brown_bat,15.8,3.9,19.7,0.023,0.30,19.0,35.0,1,1,1
7,Brazilian_tapir,5.2,1.0,6.2,160.000,169.00,30.4,392.0,4,5,4
8,Cat,10.9,3.6,14.5,3.300,25.60,28.0,63.0,1,2,1
9,Chimpanzee,8.3,1.4,9.7,52.160,440.00,50.0,230.0,1,1,1


In [3]:
animalList = df.values
columnWithoutNan = range(animalList[0].shape[0])
animalWithoutNan = range(animalList.shape[0])
for aniIdx, animal in enumerate(animalList):
    if True in pd.isnull(animal):
        for idx in range(animalList[0].shape[0]):
            if idx in columnWithoutNan:
                if pd.isnull(animal[idx]) == True:
                    columnWithoutNan.remove(idx)
        animalWithoutNan.remove(aniIdx)
print 'Number of animals without any NA column is '+str(len(animalWithoutNan))

Number of animals without any NA column is 42


I would like to use these 42 animals as a training set.
<br>
The evaluation will be given by remaining species, which contains some NA entry but not having NA feature we are looking at.
<br>
Eg. African_elephant will be used for evaluation of sleep, life, gestation, Arctic_ground_squirrel will be used for evaluation of sleep and gestation.

In [4]:
animalWithNan = range(animalList.shape[0])
for nonNan in animalWithoutNan:
    animalWithNan.remove(nonNan)
animalListForEvaluation = animalList[animalWithNan]

def extractAnimalForEvaluateSomeNa(animalList,animalWithNan, columnNum):
    evaluateFeatureList = []
    evaluateIdxList = []
    for idx, animal in enumerate(animalList):
        if idx in animalWithNan:
            if pd.isnull(animal)[columnNum] == False:
                evaluateFeatureList.append(animal[columnNum])
                evaluateIdxList.append(idx)
    return np.array(evaluateIdxList), np.array(evaluateFeatureList)

def extractTrainingSet(animalList, animalWithoutNan, columnNum):
    animalWithoutNanList = animalList[animalWithoutNan]
    trainingList = [feature[columnNum] for feature in animalWithoutNanList]
    return np.array(trainingList)

In [5]:
filteredData = np.array([animal[columnWithoutNan[1:]] for animal in animalList]).astype('float')

In [6]:
columnWithNan = range(animalList[0].shape[0])
for nonNan in columnWithoutNan:
    columnWithNan.remove(nonNan)

I would like to employ different linear regession model to see which features have higher weight in each predicting feature,
<br>
but as LASSO there is L1-norm penality, normalisation of each feature is needed.
<br>
Otherwise, weights of feature with larger number will tends to be 0.
<br>
Also, as Some number of some animals, like body of elephant is too large.
<br>
Here logistic function-based normalization is performed (similar to z-score):

In [7]:
import math
def sigmoidNormalisation(valueList, value):
    maxVal = max(valueList)
    medianVal = np.median(valueList)
    return 1/(1+math.exp(-1*(value-medianVal)/maxVal))

filteredWithoutNanAnimal = filteredData[animalWithoutNan].T
# fig = plt.figure()
# for idx, feature in enumerate(filteredWithoutNanAnimal):
#     ax = fig.add_subplot(5,1,idx+1)
#     ax.hist(feature)
# fig

normalisedAnimalList = []
for animal in filteredData:
    normalisedAnimal = []
    for idx, feature in enumerate(animal):
        normalisedAnimal.append(sigmoidNormalisation(filteredWithoutNanAnimal[idx], feature))
    normalisedAnimalList.append(normalisedAnimal)
normalisedAnimalList = np.array(normalisedAnimalList)


KFold(n_splits=10, random_state=None, shuffle=False)

In [8]:
def predictedVSActual(trainingpredicted, trainingactual, evaluatepredicted, evaluateactual, feature, regressor):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.scatter(trainingpredicted, trainingactual, label='training')
    print 'Training r2_score of '+feature+' '+regressor+'='+str(r2_score(trainingactual, trainingpredicted))
    
    ax1.scatter(evaluatepredicted, evaluateactual, label='evaluation')
    ax1.set_title('Actual VS Predicted ('+feature+'), Regressor:' + regressor)
    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('Actual')
    ax1.legend()
    print 'Evaluation r2_score of '+feature+' '+regressor+'='+str(r2_score(evaluateactual, evaluatepredicted))
    return fig, r2_score(trainingactual, trainingpredicted)

In [9]:
from sklearn.linear_model import (LinearRegression, 
                                  LassoCV, 
                                  LassoLarsIC,
                                  OrthogonalMatchingPursuitCV,
                                  BayesianRidge,
                                  ARDRegression)
from sklearn.metrics import r2_score

regressors = [('OLS', LinearRegression()),
              ('LassoCV', LassoCV(cv=20)),
              ('LassoLarsIC', LassoLarsIC(criterion='bic')),
              ('OmpCV', OrthogonalMatchingPursuitCV(cv=20)),
              ('BayesianRidge', BayesianRidge()),
              ('ARDRegression', ARDRegression())]

In [103]:
def plotCoef(coefList, regressors, r2ScoreList, featurepredicting):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    x = range(len(coefList[0]))
    xtickslabel = [df.axes[1][nonNan] for nonNan in columnWithoutNan[1:]]
    for idx, coef in enumerate(coefList):
        ax.plot(x, coef, label=regressors[idx]+' r2 = '+"%.2f"%r2ScoreList[idx], marker='o', ls='--')
        ax.set_xticks(x)
        ax.set_xticklabels(xtickslabel)
    ax.legend()
    ax.set_title('Weights of each feature correlated to '+featurepredicting)
    return fig

def SigmoidWeighted(r2ScoreList):
    sigmoidr2 = 1/(1+np.exp(-1*r2ScoreList))
    sumSigR2= sum(sigmoidr2)
    return sigmoidr2/sumSigR2

def hybridRegressor(coefList, r2ScoreList):
    sigR2weight = SigmoidWeighted(r2ScoreList)
    hybridWeighted = coefList.T.dot(sigR2weight)
    print hybridWeighted
    return hybridWeighted

def compareRegressor(featurePredicting):
    nanColumn = featurePredicting
    plt.close('all')
    coefList = []
    r2ScoreList = []
    weightedFeatureName = df.axes[1][columnWithoutNan][1:]
    fea = df.axes[1][nanColumn]
    print fea+' Regression:'
    trainingfeatureY = [feature[nanColumn] for feature in animalList[animalWithoutNan]]
    trainingfeatureX = normalisedAnimalList[animalWithoutNan]
    evaluateIdxList, evaluateFeatureList = extractAnimalForEvaluateSomeNa(animalList,animalWithNan, nanColumn)
    for name, regressor in regressors:
        model = regressor
        model.fit(trainingfeatureX, trainingfeatureY)
        predicted = model.predict(trainingfeatureX)
        evaluatingPrediction = model.predict(normalisedAnimalList[evaluateIdxList])
        fig, r2_score_training = predictedVSActual(predicted, trainingfeatureY, evaluatingPrediction, evaluateFeatureList, fea, name)
        print 'Weights of regressed by '+name
        print weightedFeatureName
        print model.coef_
        coefList.append(np.array(model.coef_).astype('float'))
        r2ScoreList.append(r2_score_training)
    coefList = np.array(coefList)
    r2ScoreList = np.array(r2ScoreList)
    regressorsName = [name for name, regressor in regressors]
    
    hybridWeight = hybridRegressor(coefList, r2ScoreList)
    
    hybridtrainingY = trainingfeatureX.dot(hybridWeight)
    hybridEvaluateY = normalisedAnimalList[evaluateIdxList].dot(hybridWeight)
    fig, r2_score_training = predictedVSActual(hybridtrainingY, trainingfeatureY, hybridEvaluateY, evaluateFeatureList, fea, 'hybridWeight')
    coefList = np.append(coefList, [hybridWeight])
    print coefList
    regressorsName.append('hybrid')
    r2ScoreList = np.append(r2ScoreList, [r2_score_training])
    plotCoef(coefList, regressorsName, r2ScoreList, fea)
    hybridWeight
    

In [104]:
# UI stuff
from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""
display(Javascript(disable_js))

featureDict = {}
for nanColumn in columnWithNan:
    featureDict[df.axes[1][nanColumn]] = nanColumn

interactPlot = interactive(compareRegressor, featurePredicting=featureDict)
output = interactPlot.children[-1]
display(interactPlot)

<IPython.core.display.Javascript object>